# Here, poc means proof of concept.

In [1]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# use word vector to calculate the similarity of 2 sentences.
def calculate_recognize_score(sentence_1:str, sentence_2:str) -> float:
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform([sentence_1, sentence_2])
    cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
    similarity_score = cosine_sim[0][0] * 100
    return round(similarity_score, 2)


import spacy
def preprocess(text:str) -> str:
    text = text.lower()
    text = text.replace("\n", " ")
    
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    processed_text = ' '.join([token.lemma_ for token in doc])
    
    return processed_text

In [3]:
sentence = """The Renaissance was a period of cultural, artistic, political, and economic rebirth in Europe following the Middle Ages.

The Renaissance generally took place from the 14th century to the 17th century.

The Renaissance promoted the rediscovery of classical philosophy, literature, and art.

The Renaissance is credited with bridging the gap between the Middle Ages and modern civilization.

The Renaissance saw the rise of great thinkers, authors, statesmen, scientists, and artists.

The Renaissance was marked by global exploration, opening new lands and cultures to European commerce.

The Renaissance began in Italy, specifically in Florence.

The Medici family of Florence played a crucial role in funding the Renaissance.

Humanism was a cultural movement that gained momentum during the Renaissance.

The Renaissance encouraged the idea that man was the center of his own universe.

The Renaissance promoted achievements in education, classical arts, literature, and science.

The invention of the Gutenberg printing press in 1450 was a key factor in spreading Renaissance ideas.

Renaissance scholars like Francesco Petrarch and Giovanni Boccaccio promoted the renewal of traditional Greek and Roman culture.

The Renaissance spread from Italy to France and throughout Europe in the 15th century.

Renaissance art was characterized by realism and naturalism.

Artists during the Renaissance used techniques like perspective, shadows, and light to add depth to their work.

Leonardo da Vinci was one of the most famous Renaissance figures, known for painting "The Mona Lisa."

Michelangelo, another Renaissance giant, is celebrated for his sculptures and the Sistine Chapel.

Raphael, a Renaissance painter, is best known for his work "The School of Athens."

Renaissance architects like Filippo Brunelleschi studied mathematics to design large buildings with domes.

Galileo, a key figure of the Renaissance, revolutionized astronomy with his telescope observations.

Nicolaus Copernicus, during the Renaissance, proposed the heliocentric theory of the solar system.

The Renaissance period saw advances in science, with figures like Rene Descartes and Galileo shaping modern thought.

Desiderius Erasmus was a prominent Renaissance scholar who defined the humanist movement in Northern Europe.

The Renaissance was influenced by increased trade and finance, impacting the cultural shifts in Europe.

Renaissance artists aimed to depict people and objects in a more lifelike and emotionally resonant way.

The Renaissance had a profound impact on European politics, economics, and social structures.

During the Renaissance, writers like Geoffrey Chaucer contributed to literature with works like "The Canterbury Tales."

The Renaissance marked the decline of feudalism and the rise of a more market-driven economy.

Exploration during the Renaissance led to the discovery of new lands and trading routes.

Christopher Columbus, Ferdinand Magellan, and Amerigo Vespucci were key explorers of the Renaissance era.

The Protestant Reformation, led by Martin Luther, emerged during the Renaissance as people questioned the Catholic Church.

William Shakespeare, a product of the Renaissance, became one of the most famous playwrights of all time.

Renaissance thinkers like Niccolo Machiavelli influenced political philosophy with works like "The Prince."

Renaissance scholars encouraged the study of classical Greek and Roman texts.

The Renaissance brought about a renewed interest in the human body, leading to anatomical studies by artists like da Vinci.

Sandro Botticelli, a Renaissance painter, is known for works such as "The Birth of Venus."

During the Renaissance, the Council of Trent and the Roman Inquisition sought to suppress ideas that challenged the Catholic Church.

Renaissance intellectuals blended artistic, scientific, and philosophical disciplines in their works.

The Renaissance was responsible for breaking down medieval ideas and encouraging modern scientific thinking.

Renaissance sculpture, exemplified by Donatello's work, emphasized naturalism and lifelike forms.

The Renaissance marked a turning point in European education, as universities became centers for new ideas.

Advances in navigation during the Renaissance led to increased European exploration and colonization.

Renaissance architecture, such as St. Peter's Basilica, combined classical elements with modern engineering.

During the Renaissance, musicians like William Byrd advanced the art of composition, particularly with madrigals.

The Renaissance inspired the creation of iconic works such as Michelangelo's "David" and da Vinci's "The Last Supper."

Renaissance humanism influenced modern secularism and the importance placed on human achievements.

The Renaissance led to the development of new artistic techniques, including chiaroscuro, which enhanced the realism of paintings.

The Renaissance produced some of the most famous works of literature, including Dante's "Divine Comedy."

Renaissance scholars like Thomas Hobbes contributed to the development of political philosophy and social contract theory.

The Renaissance promoted the idea that learning and knowledge could be pursued for personal fulfillment.

During the Renaissance, artists and scientists often worked closely together, as seen in da Vinci's studies of anatomy.

The Renaissance helped establish the basis for the scientific method, which emphasized observation and experimentation.

The Renaissance was a period of increased artistic patronage, with wealthy individuals and families funding artists.

The Renaissance influenced modern Western thought, especially in the areas of philosophy, art, and science.

Renaissance exploration led to the exchange of goods, ideas, and cultures between Europe and the Americas.

Humanism during the Renaissance encouraged individuals to question established religious authorities and doctrines.

The Renaissance fostered the creation of new intellectual movements, such as the Enlightenment.

Renaissance artists often used their work to explore religious themes, as seen in works like Raphael's \"Madonna.\""""

# Approaches

#### Approach 1

In [4]:

ans_list = [preprocess(item) for item in sentence.split("\n") if item != ""]
question = "When does was the Renaissance happen?"

new_list = ans_list.copy()
# sort best 5 answers
sorted_list = sorted(new_list, key=lambda x: calculate_recognize_score(preprocess(question), x), reverse=True)
for i in range(5):
    print(sorted_list[i])
    print(calculate_recognize_score(question, sorted_list[i]))
    print("\n\n\n")

the renaissance encourage the idea that man be the center of his own universe .
23.3




the renaissance be credit with bridge the gap between the middle age and modern civilization .
22.57




the renaissance be influence by increase trade and finance , impact the cultural shift in europe .
17.8




the renaissance promote the idea that learning and knowledge could be pursue for personal fulfillment .
17.8




humanism be a cultural movement that gain momentum during the renaissance .
15.06






#### Approach 2 (using sentence_transformers)

In [12]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

question = "What is Renaissance?"

tokenizer = AutoTokenizer.from_pretrained('/Users/lipeihong/Desktop/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('/Users/lipeihong/Desktop/all-MiniLM-L6-v2')

embeddings = []
for sentence in ans_list:
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    embeddings.append(outputs.last_hidden_state[:, 0, :])

question_embedding = model(**tokenizer(question, return_tensors="pt", padding=True, truncation=True)).last_hidden_state[:, 0, :]

In [13]:
# find top 5 answers from the list
similarity_scores = F.cosine_similarity(torch.cat(embeddings), question_embedding, dim=1)
sorted_list = sorted(zip(ans_list, similarity_scores), key=lambda x: x[1], reverse=True)

for i in range(5):
    print(sorted_list[i][0])
    print(sorted_list[i][1])
    print("\n\n\n")

the renaissance be a period of cultural , artistic , political , and economic rebirth in europe follow the middle age .
tensor(0.9189, grad_fn=<UnbindBackward0>)




the renaissance begin in italy , specifically in florence .
tensor(0.9176, grad_fn=<UnbindBackward0>)




renaissance intellectual blend artistic , scientific , and philosophical discipline in their work .
tensor(0.9158, grad_fn=<UnbindBackward0>)




the renaissance promote the idea that learning and knowledge could be pursue for personal fulfillment .
tensor(0.9070, grad_fn=<UnbindBackward0>)




the renaissance promote achievement in education , classical art , literature , and science .
tensor(0.8989, grad_fn=<UnbindBackward0>)




